In [ ]:
# @title
current_dir = "gdrive/My Drive/Colab Notebooks/Thesis/gold_standard"

import os
import glob
from google.colab import drive
import pandas as pd
import re
import numpy as np
from sklearn.svm import SVR
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
import pickle
from collections import Counter


# Import SVC with a linear kernel
from sklearn.svm import SVC
linear_svc = SVC(kernel='linear')


# Import Logistic Regression
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression()

# Import Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()

# Import Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB
naive_bayes = GaussianNB()

from sklearn.neighbors import KNeighborsClassifier

# Mount Google Drive
drive.mount('/content/gdrive')

os.chdir(current_dir)

In [ ]:
# @title
ls

In [ ]:
# @title
with open('enriched_dataframe.pkl', 'rb') as file:
    data  = pickle.load(file)

train_data = data

for i in train_data["clean_text"]:
  print(i)

 my very first contemplation of criminal behavior was trying to uh put together a plan of getting away with murder and i was gonna kill my brother i knew that every sunday id jump on his bike and hed head down the road to join up with some other fellas and he loved that bike and he used to tell me you ever touch my bike ill kill you so this sunday morning while hes working i take his bike i knew if i had provoked it just a little bit more that he would chase me so i gave him a couple of kicks and took off on a dead run for a tree that was right beside my grandfathers barn and it went up onto the roof and on the opposite side of the barn there were cutter rakes and sure enough he chased me up there and as he uh topped edge of that roof i kicked him and he fell onto those rakes some court people came got me i stabbed my first man um i uh stabbed him uh he lived but it sent out a word a clear word to the rest of them that uh you dont wanna be messing with this kid hell stick ya this parti

In [ ]:
# @title
#four example vectorizers (to see which features are active)
vectorizers = [CountVectorizer(ngram_range=(1,1)), TfidfVectorizer(ngram_range=(1,1)), CountVectorizer(ngram_range=(2,2)), TfidfVectorizer(ngram_range=(2,2))]

In [ ]:
# @title
train_data.head() #sanity check

,name,score,text,clean_text,binary,trinary,ADJ,ADP,ADV,AUX,...,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X,SPACE
0,wayne,40.0,\nmy very first contemplation of criminal beha...,my very first contemplation of criminal behav...,psychopathic,high,first criminal other little more dead opposite...,of of with on down up with of off on for besid...,very together away ever so just so right sure ...,was was s had would was do be is m are can be ...,...,five twenty four four twelve forty five twenty,to na to to nt to to to to to to not to nt to ...,my i my i his he he he me you my you he i his ...,sunday i d d sunday january john lennon nelson,,that while if that as that when that that so t...,,trying put getting gon kill knew head join lov...,,
1,carl,36.0,I had a statutory rape on a minor fifteen uh t...,i had a statutory rape on a minor fifteen uh t...,psychopathic,high,statutory minor sexual female old good female ...,on for on on around of to at,then someday very at around,was was be is is m be,...,four one,not to na to,i i a she its me it my i i i who i,,,for whether that when that where,,had arrested aggravated be re return gon have m m,,
2,bundy,34.0,\nyes that's true\n\ni grew up in a wonderful ...,yes thats true i grew up in a wonderful home ...,psychopathic,high,true wonderful dedicated young young local loc...,up in with of as of outside in to in of out of...,certainly again more generally more more more ...,do would re might would is would is will is wa...,...,two one five twelve thirteen one twenty thirty...,to nt to to to to nt nt nt nt to not not nt nt...,that i i i what i i i you we our whatever they...,dobson i d god god john q god dobson,,that as that as than once until where that whe...,x,s grew loving mean encountered call think expl...,a,
3,bernardo,35.0,\npaul jason teal aka paul bernardo\n\n\nyeah ...,paul jason teal aka paul bernardo yeah im and...,psychopathic,high,peel regional public peel regional big crazy c...,about with of on for by around on to on with f...,aka forward maybe here just in back there here...,did was did ve been was did did were can was a...,...,seventeen one one one two,nt to not nt nt na to to nt nt nt nt not to no...,i it what that you i i there any i i they they...,paul jason teal paul bernardo september septem...,,that that that that why why if if that that if...,,m makes thought came thought do turned waited ...,,
4,dahmer,23.0,\nhey you're lucky you came up on a\n\n\nday w...,hey youre lucky you came up on a day when the...,not psychopathic,medium,lucky crazy slow steady ordinary last last int...,up on like out of to about to of of off for of...,billy up very just just very so thoroughly jus...,was was was were were was do been been be is i...,...,fourteen fifteen fifteen three eighteen five n...,nt to to not not to not nt nt nt nt to to to n...,you you there it nothing nothing i my it me it...,downtown milwaukee american indian,,when as because how why that why why how where...,,re came s snowing read sent spent reading read...,,


Experiment 1: Unigrams, CountVectorizer()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Assuming 'data' is your DataFrame and it has columns 'clean_text' and 'binary'
X = data['clean_text']
y = data['binary']

# Initialize CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1))

# Transform the text data to numeric format
X_transformed = vectorizer.fit_transform(X)

# Initialize Leave-One-Out Cross-Validation
loo = LeaveOneOut()

# Lists to store predictions, actual values, and coefficients
y_true, y_pred = [], []
coefficients_list = []

# Loop over each split
for train_index, test_index in loo.split(X_transformed):
    X_train, X_test = X_transformed[train_index], X_transformed[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize the classifier with Logistic Regression for each split
    classifier = LogisticRegression()

    # Fit the model
    classifier.fit(X_train, y_train)

    # Predict on the test set
    prediction = classifier.predict(X_test)

    # Store the true and predicted values
    y_true.extend(y_test)
    y_pred.extend(prediction)

    # Store the coefficients
    coefficients_list.append(classifier.coef_[0])

# Calculate the accuracy and print the classification report
accuracy = accuracy_score(y_true, y_pred)
print(f"LOOCV Accuracy: {accuracy}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

# Calculate the average of the coefficients across all CV splits
average_coefficients = np.mean(coefficients_list, axis=0)

# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Pair the average coefficients with feature names
features_and_average_coefficients = zip(feature_names, average_coefficients)

# Sort features by their average coefficients, in descending order of absolute value
sorted_features = sorted(features_and_average_coefficients, key=lambda x: abs(x[1]), reverse=True)

# Print feature names and their average coefficients in sorted order
print("\nFeature Names and their Averaged Coefficients Over 10 Splits (in decreasing order of absolute value):")
for feature, coef in sorted_features[0:50]:
    print(f"{feature}: {coef}")


LOOCV Accuracy: 0.8

Classification Report:
                  precision    recall  f1-score   support

not psychopathic       0.75      0.75      0.75         4
    psychopathic       0.83      0.83      0.83         6

        accuracy                           0.80        10
       macro avg       0.79      0.79      0.79        10
    weighted avg       0.80      0.80      0.80        10


Feature Names and their Averaged Coefficients Over 10 Splits (in decreasing order of absolute value):
was: -0.08958271065705885
it: -0.06555247858779564
the: -0.047930903679439175
you: 0.0454667376644564
my: -0.04383308285199304
its: 0.03832007947455911
uh: -0.03566136458941703
had: -0.03276016376459285
to: -0.03198143958076482
on: -0.03063068452726463
her: -0.02882707406381312
and: -0.027077561491729636
have: 0.024766521118194905
of: 0.02346677645968168
there: -0.02118786154652133
out: -0.020250035568280193
she: -0.020090300381391153
up: -0.018894786906323707
know: 0.018829368656470842
are: 0.016

Experiment 2: Bigrams, CountVectorizer()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Assuming 'data' is your DataFrame and it has columns 'clean_text' and 'binary'
X = data['clean_text']
y = data['binary']

# Initialize CountVectorizer
vectorizer = CountVectorizer(ngram_range=(2, 2))

# Transform the text data to numeric format
X_transformed = vectorizer.fit_transform(X)

# Initialize Leave-One-Out Cross-Validation
loo = LeaveOneOut()

# Lists to store predictions, actual values, and coefficients
y_true, y_pred = [], []
coefficients_list = []

# Loop over each split
for train_index, test_index in loo.split(X_transformed):
    X_train, X_test = X_transformed[train_index], X_transformed[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize the classifier with Logistic Regression for each split
    classifier = LogisticRegression()

    # Fit the model
    classifier.fit(X_train, y_train)

    # Predict on the test set
    prediction = classifier.predict(X_test)

    # Store the true and predicted values
    y_true.extend(y_test)
    y_pred.extend(prediction)

    # Store the coefficients
    coefficients_list.append(classifier.coef_[0])

# Calculate the accuracy and print the classification report
accuracy = accuracy_score(y_true, y_pred)
print(f"LOOCV Accuracy: {accuracy}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

# Calculate the average of the coefficients across all CV splits
average_coefficients = np.mean(coefficients_list, axis=0)

# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Pair the average coefficients with feature names
features_and_average_coefficients = zip(feature_names, average_coefficients)

# Sort features by their average coefficients, in descending order of absolute value
sorted_features = sorted(features_and_average_coefficients, key=lambda x: abs(x[1]), reverse=True)

# Print feature names and their average coefficients in sorted order
print("\nFeature Names and their Averaged Coefficients Over 10 Splits (in decreasing order of absolute value):")
for feature, coef in sorted_features[0:50]:
    print(f"{feature}: {coef}")


LOOCV Accuracy: 0.6

Classification Report:
                  precision    recall  f1-score   support

not psychopathic       0.00      0.00      0.00         4
    psychopathic       0.60      1.00      0.75         6

        accuracy                           0.60        10
       macro avg       0.30      0.50      0.37        10
    weighted avg       0.36      0.60      0.45        10


Feature Names and their Averaged Coefficients Over 10 Splits (in decreasing order of absolute value):
it was: -0.12482158368067282
and uh: -0.0594475768179071
in the: -0.05826622159941448
on the: -0.034931901027998764
the the: -0.03304816038996898
it it: -0.031195494388034345
there was: -0.02762757121132372
to the: -0.027315045011331484
my mother: -0.02508770420765617
and and: -0.023873440220119495
the car: -0.02302067729643068
the first: -0.02240776715599985
you guys: 0.022330238206916635
of my: -0.02199614600464253
was uh: -0.02193765330585699
at the: -0.02190100462221465
the door: -0.0215567843

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Experiment 3: Bigrams, TfidfVectorizer()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Assuming 'data' is your DataFrame and it has columns 'clean_text' and 'binary'
X = data['clean_text']
y = data['binary']

# Initialize CountVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 1))

# Transform the text data to numeric format
X_transformed = vectorizer.fit_transform(X)

# Initialize Leave-One-Out Cross-Validation
loo = LeaveOneOut()

# Lists to store predictions, actual values, and coefficients
y_true, y_pred = [], []
coefficients_list = []

# Loop over each split
for train_index, test_index in loo.split(X_transformed):
    X_train, X_test = X_transformed[train_index], X_transformed[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize the classifier with Logistic Regression for each split
    classifier = LogisticRegression()

    # Fit the model
    classifier.fit(X_train, y_train)

    # Predict on the test set
    prediction = classifier.predict(X_test)

    # Store the true and predicted values
    y_true.extend(y_test)
    y_pred.extend(prediction)

    # Store the coefficients
    coefficients_list.append(classifier.coef_[0])

# Calculate the accuracy and print the classification report
accuracy = accuracy_score(y_true, y_pred)
print(f"LOOCV Accuracy: {accuracy}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

# Calculate the average of the coefficients across all CV splits
average_coefficients = np.mean(coefficients_list, axis=0)

# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Pair the average coefficients with feature names
features_and_average_coefficients = zip(feature_names, average_coefficients)

# Sort features by their average coefficients, in descending order of absolute value
sorted_features = sorted(features_and_average_coefficients, key=lambda x: abs(x[1]), reverse=True)

# Print feature names and their average coefficients in sorted order
print("\nFeature Names and their Averaged Coefficients Over 10 Splits (in decreasing order of absolute value):")
for feature, coef in sorted_features[0:50]:
    print(f"{feature}: {coef}")


LOOCV Accuracy: 0.6

Classification Report:
                  precision    recall  f1-score   support

not psychopathic       0.00      0.00      0.00         4
    psychopathic       0.60      1.00      0.75         6

        accuracy                           0.60        10
       macro avg       0.30      0.50      0.37        10
    weighted avg       0.36      0.60      0.45        10


Feature Names and their Averaged Coefficients Over 10 Splits (in decreasing order of absolute value):
was: -0.28407420157386226
uh: -0.2446324788443785
it: -0.21850204551571
you: 0.194324940752507
her: -0.1673944922972616
and: -0.15172589462340785
its: 0.13144244757850992
the: -0.12091474945646612
my: -0.11727423522469764
im: 0.11695327344964039
child: 0.1121388316640535
there: -0.08754158038760304
had: -0.08615258888066632
in: -0.08455261276680874
she: -0.08334818945261561
is: 0.08157716670304005
sex: -0.08081420836366635
um: -0.07455557892961735
gun: -0.07315261451232077
female: 0.07176242466113

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Experiment 4: Bigrams, TfidfVectorizer()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Assuming 'data' is your DataFrame and it has columns 'clean_text' and 'binary'
X = data['clean_text']
y = data['binary']

# Initialize CountVectorizer
vectorizer = TfidfVectorizer(ngram_range=(2, 2))

# Transform the text data to numeric format
X_transformed = vectorizer.fit_transform(X)

# Initialize Leave-One-Out Cross-Validation
loo = LeaveOneOut()

# Lists to store predictions, actual values, and coefficients
y_true, y_pred = [], []
coefficients_list = []

# Loop over each split
for train_index, test_index in loo.split(X_transformed):
    X_train, X_test = X_transformed[train_index], X_transformed[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize the classifier with Logistic Regression for each split
    classifier = LogisticRegression()

    # Fit the model
    classifier.fit(X_train, y_train)

    # Predict on the test set
    prediction = classifier.predict(X_test)

    # Store the true and predicted values
    y_true.extend(y_test)
    y_pred.extend(prediction)

    # Store the coefficients
    coefficients_list.append(classifier.coef_[0])

# Calculate the accuracy and print the classification report
accuracy = accuracy_score(y_true, y_pred)
print(f"LOOCV Accuracy: {accuracy}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

# Calculate the average of the coefficients across all CV splits
average_coefficients = np.mean(coefficients_list, axis=0)

# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Pair the average coefficients with feature names
features_and_average_coefficients = zip(feature_names, average_coefficients)

# Sort features by their average coefficients, in descending order of absolute value
sorted_features = sorted(features_and_average_coefficients, key=lambda x: abs(x[1]), reverse=True)

# Print feature names and their average coefficients in sorted order
print("\nFeature Names and their Averaged Coefficients Over 10 Splits (in decreasing order of absolute value):")
for feature, coef in sorted_features[0:50]:
    print(f"{feature}: {coef}")


LOOCV Accuracy: 0.6

Classification Report:
                  precision    recall  f1-score   support

not psychopathic       0.00      0.00      0.00         4
    psychopathic       0.60      1.00      0.75         6

        accuracy                           0.60        10
       macro avg       0.30      0.50      0.37        10
    weighted avg       0.36      0.60      0.45        10


Feature Names and their Averaged Coefficients Over 10 Splits (in decreasing order of absolute value):
it was: -0.20176823133557562
you guys: 0.11307533827306429
the lord: 0.08521374410107538
and uh: -0.08122882256187536
and um: -0.07400877637703866
female child: 0.07005111242929296
my id: -0.06475767932990883
my son: -0.06475767932990883
in the: -0.06418827074211932
my mother: -0.06356288636525392
im not: 0.06292726337832688
its its: 0.06251839928432502
we have: 0.06104544642813251
was uh: -0.058780045497090774
the the: -0.05671823798557072
to uh: -0.055875090217344825
it it: -0.055105404952270044

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
